<a href="https://colab.research.google.com/github/Mustafaadeel1/Assignament-Q2/blob/main/%22Assignment_05_Exploring_Gemini_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import userdata
import os
GEMINI_API_KEY = userdata.get('GOOGLE_API_KEY')
os.environ['GEMINI_API_KEY'] = GEMINI_API_KEY


In [2]:
!pip install --upgrade --quiet google-genai

In [3]:
!pip install gtts


In [4]:

from IPython.display import display, Audio ,Markdown

In [5]:
from google import genai
from google.genai import Client

client: Client = genai.Client(
    api_key=GEMINI_API_KEY,
)

model: str = "gemini-2.0-flash-exp"

# **1. Uploading file**

In [9]:
from google.colab import files

# Upload video file
uploaded = files.upload()

# List uploaded files
for filename in uploaded.keys():
    print(f'Uploaded file: {filename}')


Saving WIN_20241227_19_19_34_Pro.mp4 to WIN_20241227_19_19_34_Pro.mp4
Uploaded file: WIN_20241227_19_19_34_Pro.mp4


## **2. Converting file into readable format**

In [10]:
import time

def upload_video(video_file_name):
  video_file = client.files.upload(path="WIN_20241227_19_19_34_Pro.mp4")
  while video_file.state == "PROCESSING":
      print('Waiting for video to be processed.')
      time.sleep(10)
      video_file = client.files.get(name=video_file.name or "")

  if video_file.state == "FAILED":
    raise ValueError(video_file.state)
  print(f'Video processing complete: ' + (video_file.uri or ""))

  return video_file

pottery_video = upload_video('WIN_20241227_19_19_34_Pro.mp4')

Waiting for video to be processed.
Video processing complete: https://generativelanguage.googleapis.com/v1beta/files/vjnigc1j7uey


# **3. Giving video to gemini for scenes descriptions**

In [11]:

from google.genai.types import Content, Part
prompt = """For each scene in this video,
            generate captions that describe the scene.
            Place each caption into an object with the timecode of the caption in the video.
         """

video = pottery_video

response = client.models.generate_content(
    model=model,
    contents=[
        Content(
            role="user",
            parts=[
                Part.from_uri(
                    file_uri=video.uri or "",
                    mime_type=video.mime_type or ""),
                ]),
        prompt,
    ]
)
Scenes = response.text
Markdown(Scenes)


```json
[
  {
    "timecode": "00:00",
    "caption": "A young man with dark hair and a mustache is looking directly at the camera while wearing glasses. He is in a room with a striped wall and a decorative wooden headboard."
  },
  {
      "timecode":"00:02",
      "caption":"The young man's expression remains neutral as he looks towards the camera, a wooden cabinet can be seen behind him."
  },
    {
      "timecode":"00:06",
      "caption":"The man continues to look at the camera, his head slightly tilted. He is in a softly lit environment with furniture visible in the background."
   },
  {
    "timecode": "00:10",
   "caption": "The young man looks to his left, then back at the camera, maintaining his calm expression."
    },
  {
    "timecode": "00:12",
    "caption": "The man appears to be paying attention to something off-camera, before looking back to the center."
  },
  {
    "timecode": "00:15",
     "caption": "He looks directly at the camera once more, still in the same indoor location."
    }
]
```

# **4 . Giving video to gemini for audio analysis**

In [12]:

from gtts import gTTS
from google.genai.types import Content, Part


# Your prompt
prompt = """For each scene in this video,
            generate captions that describe the audio along with any spoken text placed in quotation marks.
            Place each caption into an object with the timecode of the caption in the video.
         """

video = pottery_video

# Generate response
response = client.models.generate_content(
    model=model,
    contents=[
        Content(
            role="user",
            parts=[
                Part.from_uri(
                    file_uri=video.uri or "",
                    mime_type=video.mime_type or ""),
            ]),
        prompt,
    ]
)

# Get the text response
output_text = response.text

# Print and speak the output
print(output_text)

```json
{
    "00:00": {
        "caption": "There is only speech present in the audio"
    },
    "00:03": {
        "caption": "There is only speech present in the audio"
     },
       "00:06": {
        "caption": "There is only speech present in the audio"
    },
        "00:09": {
        "caption": "There is only speech present in the audio"
    },
          "00:11": {
        "caption": "There is only speech present in the audio"
    },
            "00:13": {
        "caption": "There is only speech present in the audio"
    },
            "00:15": {
        "caption": "There is only speech present in the audio"
    }
}
```


# **5. Converting Gemini text responsw to audio using gTTS python library**

In [13]:
# Function to play audio
def speak_output(message):
    tts = gTTS(text=message, lang='en')
    tts.save("response.mp3")
speak_output(output_text)


In [14]:


display(Audio("response.mp3", autoplay=True))


# **6.Chatting with gemini on the basis on video**

In [15]:


from google.genai.chats import Chat

# Initiate a chat
chat: Chat = client.chats.create(
    model=model
)

# Send chat a message
response = chat.send_message(output_text)
Markdown(response.text)


```json
{
    "00:00": {
        "caption": "There is only speech present in the audio."
    },
    "00:03": {
        "caption": "There is only speech present in the audio."
     },
       "00:06": {
        "caption": "There is only speech present in the audio."
    },
        "00:09": {
        "caption": "There is only speech present in the audio."
    },
          "00:11": {
        "caption": "There is only speech present in the audio."
    },
            "00:13": {
        "caption": "There is only speech present in the audio."
    },
            "00:15": {
        "caption": "There is only speech present in the audio."
    }
}
```


In [16]:


response = chat.send_message("What is my person name in the video?")
Markdown(response.text)


I'm sorry, but I have no way of knowing who you are or what your name is based on the information I have. The transcript only indicates that there is speech present, but it doesn't give any details about the speaker or speakers. 

To figure out your name, you'd need to provide more context or additional information, such as:

*   **The actual video**: If you can share the video (or a link to it), I might be able to identify you from visual or auditory cues.
*   **Any previous interactions with me**: If you've provided me with your name in a previous conversation, I might remember it.
*   **Additional information about the video:** If you can tell me anything else about the content of the video, such as the topic or other participants, it might help.

Without more information, I am unable to tell you who is speaking, sorry.


In [17]:
response = chat.send_message("What is my person experties in the video?")
Markdown(response.text)

Okay, I understand. Based on the provided transcript alone, which simply states "There is only speech present in the audio" at each timestamp, **I have absolutely no information about your expertise in the video.** 

The transcript only tells us *that* someone is speaking, not *what* they are speaking about or their level of knowledge on any given subject.

To determine your area of expertise in the video, I would need:

1.  **The actual content of the speech:** I need to know what is being said in the video. The words spoken would indicate the topic(s) and the speaker's level of understanding.
2.  **Context about the video:** Knowing the overall purpose of the video (e.g., is it an instructional video, a debate, a news segment, etc.) would give me clues about the expected areas of expertise.
3.  **Information about you:** If you have context from previous interactions with me or information you'd like to provide, then this could also help me to understand your expertise.

Without any of this, it's impossible for me to infer anything about your expertise.

So, in short, based on the given transcript, **I can't tell you anything about your expertise in the video.** I need more information.
